In [131]:
import pandas as pd
import geopandas as gpd
import folium
from sklearn.cluster import *

CSV_PATH="../data/processed/edit_straatvinken_hackathon_20220325.csv"

# read coordinates for inference
df = pd.read_csv(CSV_PATH)
gpdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.long, df.lat), crs=4326)
print(gpdf.shape)
gpdf[["lat", "long"]]

(3550, 290)


,lat,long
0,50.719737,4.258857
1,50.728366,4.243156
2,50.734949,4.232230
3,50.739619,3.605092
4,50.741474,3.594312
...,...,...
3545,51.468886,4.454877
3546,51.469910,4.465937
3547,51.470587,4.475584
3548,51.471278,4.476050


In [134]:
import numpy as np

4.605170185988092

In [ ]:
np.power()

In [156]:
agg_cols_green = ["lu_park_prop", "lu_grass_prop", "lu_meadow_prop"]
agg_col_greenname = "greenstuff"
agg_cols_red = ["lu_industrial_prop", "lu_commercial_prop", "lu_residential_prop"]
agg_col_redname = "deathstuff"
gpdf[agg_greenname] = gpdf[agg_col_green].sum(axis=1)
gpdf[agg_redname] = gpdf[agg_col_red].sum(axis=1)

cols = ["lat", "long", "walk", "bike", "car", "van", "truck", "geometry"]
cols.append(agg_col_greenname)
cols.append(agg_col_redname)
m=folium.Map(location=[51, 4.45], tiles="CartoDb positron")
gpdf[gpdf.eval(f"{agg_col_greenname}>.3")][cols].explore(column = agg_col_greenname, marker_kwds={"radius": 1, "opacity":0.5}, tiles="CartoDb positron", cmap="Greens", m=m)
gpdf[gpdf.eval(f"{agg_col_redname}>.7")][cols].explore(column = agg_col_redname, marker_kwds={"radius": 1, "opacity":0.5}, tiles="CartoDb positron", cmap="Reds", m=m)

n_clusters_green = 10
n_clusters_red = 10
gclust = KMeans(n_clusters=n_clusters_green, random_state=42)
rclust = KMeans(n_clusters=n_clusters_red, random_state=42)
#green atol clusters
green_df = gpdf[gpdf.eval(f"{agg_col_greenname} > 0.3")][["lat", "long", agg_col_greenname]]
#print(green_df.shape)
red_df = gpdf[gpdf.eval(f"{agg_col_redname} > 0.7")][["lat", "long", agg_col_redname]]
#print(red_df.shape)

green_clusters = gclust.fit(green_df[["lat", "long"]], sample_weight=green_df[agg_col_greenname])
red_clusters = rclust.fit(red_df[["lat", "long"]], sample_weight=red_df[agg_col_redname])
gpdf["red_cluster"] = rclust.predict(gpdf[["lat", "long"]])
gpdf["green_cluster"] = gclust.predict(gpdf[["lat", "long"]])
red_clust_vcs = gpdf["red_cluster"].value_counts().reset_index()
red_clust_vcs = red_clust_vcs.sort_values(by="index")
green_clust_vcs = gpdf["green_cluster"].value_counts().reset_index()
green_clust_vcs = green_clust_vcs.sort_values(by="index")
green_atols = pd.DataFrame(green_clusters.cluster_centers_)
green_atols["atol_name"] = pd.Series(["green atol "]*10) + (green_atols.index + 1).astype(str)
green_atols["color"] = "green"
green_atols["size"] = green_clust_vcs["green_cluster"]
green_atols["icon"] = 'glyphicon-leaf'
red_atols = pd.DataFrame(red_clusters.cluster_centers_)
red_atols["atol_name"] = pd.Series(["red atol "]*10) + (red_atols.index + 1).astype(str)
red_atols["color"] = "red"
red_atols["size"] = red_clust_vcs["red_cluster"]
red_atols["icon"] = 'glyphicon-fire'
all_clusters = pd.concat([red_atols, green_atols], axis=0)
all_clusters.columns = ["long", "lat", "atol", "color", "size", "icon"]
for i, row in all_clusters.iterrows():
    folium.Marker(location=[row["long"],row["lat"]], icon=folium.Icon(color=row["color"], icon=row["icon"], )).add_to(m)
    folium.CircleMarker(
        location=[row["long"],row["lat"]], 
        radius=np.power(np.log2(row["size"]), 2)/3,  
        fill=True, fill_color=row["color"],color = 'white',fill_opacity=0.7).add_to(m)
    
   

m

In [152]:
agg_cols_green = ["lu_park_prop", "lu_grass_prop", "lu_meadow_prop"]
agg_col_greenname = "greenstuff"
agg_cols_red = ["lu_industrial_prop", "lu_commercial_prop", "lu_residential_prop"]
agg_col_redname = "deathstuff"
gpdf[agg_greenname] = gpdf[agg_col_green].sum(axis=1)
gpdf[agg_redname] = gpdf[agg_col_red].sum(axis=1)

cols = ["lat", "long", "walk", "bike", "car", "van", "truck", "geometry"]
cols.append(agg_col_greenname)
cols.append(agg_col_redname)
m=folium.Map(location=[51, 4.45], tiles="CartoDb positron")
gpdf[gpdf.eval(f"{agg_col_greenname}>.3")][cols].explore(column = agg_col_greenname, marker_kwds={"radius": 1, "opacity":0.5}, tiles="CartoDb positron", cmap="Greens", m=m)
gpdf[gpdf.eval(f"{agg_col_redname}>.7")][cols].explore(column = agg_col_redname, marker_kwds={"radius": 1, "opacity":0.5}, tiles="CartoDb positron", cmap="Reds", m=m)

n_clusters_green = 10
n_clusters_red = 10
gclust = Birch(n_clusters=n_clusters_green)
rclust = Birch(n_clusters=n_clusters_red)
#green atol clusters
green_df = gpdf[gpdf.eval(f"{agg_col_greenname} > 0.3")][["lat", "long", agg_col_greenname]]
#print(green_df.shape)
red_df = gpdf[gpdf.eval(f"{agg_col_redname} > 0.7")][["lat", "long", agg_col_redname]]
#print(red_df.shape)

green_clusters = gclust.fit(green_df[["lat", "long"]], )
red_clusters = rclust.fit(red_df[["lat", "long"]])
gpdf["red_cluster"] = rclust.predict(gpdf[["lat", "long"]])
gpdf["green_cluster"] = gclust.predict(gpdf[["lat", "long"]])
red_clust_vcs = gpdf["red_cluster"].value_counts().reset_index()
red_clust_vcs = red_clust_vcs.sort_values(by="index")
green_clust_vcs = gpdf["green_cluster"].value_counts().reset_index()
green_clust_vcs = green_clust_vcs.sort_values(by="index")
green_atols = pd.DataFrame(green_clusters.subcluster_centers_)
green_atols["atol_name"] = pd.Series(["green atol "]*green_atols.shape[0]) + (green_atols.index + 1).astype(str)
green_atols["color"] = "green"
green_atols["size"] = green_clust_vcs["green_cluster"]
green_atols["icon"] = 'glyphicon-leaf'
red_atols = pd.DataFrame(red_clusters.subcluster_centers_)
red_atols["atol_name"] = pd.Series(["red atol "]*red_atols.shape[0]) + (red_atols.index + 1).astype(str)
red_atols["color"] = "red"
red_atols["size"] = red_clust_vcs["red_cluster"]
red_atols["icon"] = 'glyphicon-fire'
all_clusters = pd.concat([red_atols, green_atols], axis=0)
all_clusters.columns = ["long", "lat", "atol", "color", "size", "icon"]
for i, row in all_clusters.iterrows():
    folium.Marker(location=[row["long"],row["lat"]], icon=folium.Icon(color=row["color"], icon=row["icon"], )).add_to(m)
    folium.CircleMarker(
        location=[row["long"],row["lat"]], 
        radius=3*np.log2(row["size"]),  
        fill=True, fill_color=row["color"],color = 'white',fill_opacity=0.7).add_to(m)
    
   

m

In [150]:
green_clust_vcs

,index,green_cluster
1,0,593
0,1,2957


In [149]:
gclust.subcluster_centers_

array([[51.04325638,  3.72115409],
       [51.20175186,  4.61443347]])

In [75]:
green_atols = pd.DataFrame(green_clusters.cluster_centers_)
green_atols["atol_name"] = pd.Series(["green atol "]*10) + (red_atols.index + 1).astype(str)
red_atols = pd.DataFrame(red_clusters.cluster_centers_)
red_atols["atol_name"] = pd.Series(["red atol "]*10) + (red_atols.index + 1).astype(str)
all_clusters = pd.concat([green_atols, red_atols], axis=0)
all_clusters.columns = ["lat", "long", "atol"]
all_clusters

,lat,long,atol
0,51.098827,4.121763,green atol 1
1,51.030931,3.732913,green atol 2
2,51.318540,4.959776,green atol 3
3,51.208112,4.421098,green atol 4
4,50.963600,3.215579,green atol 5
5,50.859337,4.771504,green atol 6
6,50.860333,4.362930,green atol 7
7,50.919216,5.226449,green atol 8
8,51.054719,4.495163,green atol 9
9,50.892309,2.896402,green atol 10


In [157]:
import dateutil

In [158]:
dateutil.__version__

'2.8.0'